<a href="https://colab.research.google.com/github/LuisIrigoyen/trabajo_final_RL/blob/pruebas_LuisIrigoyen/Modelo_15_NoisyLinear_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

- Igual que 12, cambio paramétro: dqn.compile(optimizer=Adam(learning_rate=0.0001)
- luego se modificó a dqn.compile(optimizer=Adam(learning_rate=1e-4)
- se cambió policy = BoltzmannQPolicy() a policy = EpsGreedyQPolicy(eps=0.0)
- cambio sigma_init en NoisyDense: sigma_init=0.017 a 0.05
- learning_rate aún más bajo para Adam: de 1e-4 a 5e-5

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/MODELO 15/"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

## 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/MODELO 15/  exists.

Colab: Changing directory to  /content/gdrive/My Drive/MODELO 15/
/content/gdrive/My Drive/MODELO 15
Archivos en el directorio: 
['logs', 'checkpoints', 'memory_noisy.pkl']


## 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-6n7pb52t
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-6n7pb52t
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [ ]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [ ]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

#--- nuevo -- ejfm
height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        # Convertir a float y escalar
        # Linea Agregada
        if isinstance(batch, list):
          batch = np.array(batch)

        batch = batch.astype('float32') / 255.

        # 👇 Elimina el canal innecesario si existe (forma (B, 4, 84, 84, 1))
        if batch.ndim == 5 and batch.shape[-1] == 1:
            batch = np.squeeze(batch, axis=-1)

        return batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


## 1. EJFM Implementación de la red neuronal

In [ ]:
import tensorflow as tf

class NoisyDense(tf.keras.layers.Layer):
    def __init__(self, units, sigma_init=0.05, **kwargs):
        super(NoisyDense, self).__init__(**kwargs)
        self.units = units
        self.sigma_init = sigma_init

    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        self.mu_W = self.add_weight("mu_W", shape=(input_dim, self.units),
                                    initializer='random_uniform', trainable=True)
        self.sigma_W = self.add_weight("sigma_W", shape=(input_dim, self.units),
                                       initializer=tf.keras.initializers.Constant(self.sigma_init), trainable=True)
        self.mu_b = self.add_weight("mu_b", shape=(self.units,),
                                    initializer='zeros', trainable=True)
        self.sigma_b = self.add_weight("sigma_b", shape=(self.units,),
                                       initializer=tf.keras.initializers.Constant(self.sigma_init), trainable=True)

    def call(self, x):
        epsilon_W = tf.random.normal((x.shape[-1], self.units))
        epsilon_b = tf.random.normal((self.units,))
        W = self.mu_W + self.sigma_W * epsilon_W
        b = self.mu_b + self.sigma_b * epsilon_b
        return tf.matmul(x, W) + b

    def get_config(self):
        config = super(NoisyDense, self).get_config()
        config.update({
            "units": self.units,
            "sigma_init": self.sigma_init
        })
        return config


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Permute, Convolution2D, Flatten, Activation

input_layer = Input(shape=(WINDOW_LENGTH,) + INPUT_SHAPE)
perm = Permute((2, 3, 1))(input_layer)  # Canal al final
conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(perm)
conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
conv3 = Convolution2D(64, (3, 3), strides=(1, 1), activation='relu')(conv2)
flat = Flatten()(conv3)

noisy1 = NoisyDense(512)(flat)
noisy1 = Activation('relu')(noisy1)
output_layer = NoisyDense(nb_actions)(noisy1)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 84, 84)]       0         
                                                                 
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 noisy_dense (NoisyDense)    (None, 512)               321228

In [ ]:
# Registrar la capa personalizada (una sola vez en todo el código)
from tensorflow.keras.utils import get_custom_objects
get_custom_objects().update({'NoisyDense': NoisyDense})

## 2. EJFM Implementación de la solución DQN

### Ejecución Inicial



In [ ]:
####### Solución DQN #######

import os
import pickle

# INTENTAR CARGAR LA MEMORIA
memory_file = 'memory_noisy.pkl'
if os.path.exists(memory_file):
    print("Cargando memoria desde archivo...")
    with open(memory_file, 'rb') as f:
        memory = pickle.load(f)
else:
    print("Creando nueva memoria...")
    memory = SequentialMemory(limit=1000000, window_length=4)

# Crear política como de costumbre
#policy = BoltzmannQPolicy()
policy = EpsGreedyQPolicy(eps=0.0)

# Crear agente
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=AtariProcessor(), nb_steps_warmup=50000, gamma=0.99,
               target_model_update=10000, train_interval=4, delta_clip=5)

# Compilar agente
#dqn.compile(optimizer='adam', metrics=['mae'])
#dqn.compile(optimizer=Adam(learning_rate=0.0001), metrics=['mae'])

from tensorflow.keras.optimizers.legacy import Adam     # ← incluye get_updates
# Compilar agente
dqn.compile(optimizer=Adam(learning_rate=5e-5), metrics=['mae'])

Cargando memoria desde archivo...


In [ ]:
####### Crear directorios  logs y checkpoints #######

import os
log_dir = os.path.join(drive_root, "logs")
checkpoint_dir = os.path.join(drive_root, "checkpoints")
os.makedirs(log_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Función para restaurar pesos más adelante
import re

def load_latest_weights(agent, checkpoint_dir, prefix="dqn_noisy_weights_"):
    """
    Busca el checkpoint más reciente, carga los pesos y retorna info útil.

    Parámetros:
        agent: el agente DQN ya creado y compilado.
        checkpoint_dir: carpeta con los archivos de pesos.
        prefix: prefijo de los archivos de pesos.

    Retorna:
        (ruta_archivo, pasos_entrenados) o (None, 0) si no se encuentra nada.
    """
    files = [f for f in os.listdir(checkpoint_dir) if f.endswith(".h5")]
    pattern = re.compile(rf"{re.escape(prefix)}(\d+)\.h5")

    candidates = []
    for f in files:
        match = pattern.match(f)
        if match:
            step = int(match.group(1))
            candidates.append((step, f))

    if not candidates:
        print("⚠️ No se encontraron checkpoints en:", checkpoint_dir)
        return None, 0

    # Escoger el de mayor número de pasos
    latest_step, latest_file = max(candidates, key=lambda x: x[0])
    latest_path = os.path.join(checkpoint_dir, latest_file)

    print(f"✅ Cargando pesos desde: {latest_path} (pasos entrenados: {latest_step})")
    agent.load_weights(latest_path)
    return latest_path, latest_step


In [ ]:
# Cargar pesos anteriores si existen
_, trained_steps = load_latest_weights(dqn, checkpoint_dir)

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 15/checkpoints/dqn_noisy_weights_300000.h5 (pasos entrenados: 300000)


### Primera **Ejecución**

In [ ]:
# Cargar último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)

# Entrenar agente
dqn.fit(env, nb_steps=100000, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales en formato .h5 (compatible con load_weights)
dqn.save_weights(os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5"), overwrite=True)

# GUARDAR LA MEMORIA DE EXPERIENCIA
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 15/checkpoints/dqn_noisy_weights_300000.h5 (pasos entrenados: 300000)
Training for 100000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   851/100000: episode: 1, duration: 22.121s, episode steps: 851, steps per second:  38, episode reward: 12.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 3.141 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1560/100000: episode: 2, duration: 17.543s, episode steps: 709, steps per second:  40, episode reward:  6.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 3.185 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2209/100000: episode: 3, duration: 17.059s, episode steps: 649, steps per second:  38, episode reward:  6.000, mean reward:  0.009 [ 0.000,  1.000], mean action: 3.163 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3020/100000: episode: 4, duration: 20.015s, episode steps: 811, steps per second:  41, episode reward: 10.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 3.200 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3849/100000: episode: 5, duration: 21.986s, episode steps: 829, steps per second:  38, episode reward: 12.000, mean reward:  0.014

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 50177/100000: episode: 61, duration: 33.702s, episode steps: 812, steps per second:  24, episode reward:  9.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.950 [0.000, 5.000],  loss: 0.007399, mae: 0.460153, mean_q: 0.554312
 50729/100000: episode: 62, duration: 45.413s, episode steps: 552, steps per second:  12, episode reward:  5.000, mean reward:  0.009 [ 0.000,  1.000], mean action: 2.250 [0.000, 5.000],  loss: 0.008036, mae: 0.465000, mean_q: 0.560381
 51475/100000: episode: 63, duration: 56.369s, episode steps: 746, steps per second:  13, episode reward: 17.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 1.761 [0.000, 5.000],  loss: 0.005965, mae: 0.460746, mean_q: 0.556332
 51913/100000: episode: 64, duration: 32.901s, episode steps: 438, steps per second:  13, episode reward:  3.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 2.283 [0.000, 5.000],  loss: 0.006331, mae: 0.459741, mean_q: 0.555523
 52318/100000: episode: 65, duration: 31.141s, episode s

In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 15/checkpoints/dqn_noisy_weights_400000.h5 (pasos entrenados: 400000)
Testing for 10 episodes ...
Episode 1: reward: 5.000, steps: 529
Episode 2: reward: 15.000, steps: 989
Episode 3: reward: 12.000, steps: 1107
Episode 4: reward: 5.000, steps: 345
Episode 5: reward: 11.000, steps: 628
Episode 6: reward: 18.000, steps: 979
Episode 7: reward: 9.000, steps: 625
Episode 8: reward: 5.000, steps: 428
Episode 9: reward: 7.000, steps: 458
Episode 10: reward: 23.000, steps: 1056


### Parte 2

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 15/checkpoints/dqn_noisy_weights_400000.h5 (pasos entrenados: 400000)
Training for 100000 steps ...
   523/100000: episode: 1, duration: 13.380s, episode steps: 523, steps per second:  39, episode reward: 10.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 1.996 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1179/100000: episode: 2, duration: 16.643s, episode steps: 656, steps per second:  39, episode reward:  7.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.290 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2171/100000: episode: 3, duration: 26.349s, episode steps: 992, steps per second:  38, episode reward: 21.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 2.369 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2659/100000: episode: 4, duration: 12.514s, episode steps: 488, steps per second:  39, episode reward:  6.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.158 [0.000, 5.000],  loss: 

In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

Parte 3

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)